In [17]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd 
import itertools
import re 
import time


In [ ]:
#################################################################
#Selenium para crowlear o site e fazer scrolldown automatico
################################################################

#Instancia Browser

chrome = webdriver.Chrome("chromedriver.exe")

chrome.get('https://www.quintoandar.com.br/alugar/imovel/belo-horizonte-mg-brasil/apartamento/3-quartos/de-500-a-2500-aluguel/1-vagas')
time.sleep(0.5)

body = chrome.find_element_by_tag_name("body")
no_pageDowns = 50

#Scroll Down  while no_pagedows > 0.

while no_pageDowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    no_pageDowns -= 1

fonte = chrome.page_source #Busca codigo fonte da página
soup = BeautifulSoup(fonte, "html.parser") #Faz o parser do html

chrome.quit() #Fecha janela do Browser

################################################################
# BeautifulSoup para fazer o webscraping dos dados da página
################################################################

#Variaveis procuradas na página
rua = []
bairro = []
area = []
quartos = []
aluguel = []
total = []

#Container para armazenar os dados do apto (menos preços)
aptoContainer = soup.find_all('div', class_='hhh4j4-0 hgxpIA')
if aptoContainer != []:
    
    #Manipulação dos itens da lista aptoContainer para buscar as variaveis de interesse
    for apto in aptoContainer:
        
        #Busca endereços (Rua e Bairro)
        rua_x = apto.find_all('p')[0].text #busca o elemento contendo a rua do apto x
        rua.append(rua_x)
        bairro_x = apto.find_all('p')[1].text #busca elemento contendo o bairro do apto x
        bairro.append(bairro_x)
        
        #Busca Numero de Quartos
        quartos_x = apto.find_all('span', class_='hhh4j4-1 ldRYl sc-bdVaJa hfjSkg')[1].text
        quartos_x = int(re.search(r'\d+', quartos).group()) #torna o elemento quartos legível
        quartos.append(quartos_x)
        
        #Busca area
        area_x = apto.find_all('span',class_= 'hhh4j4-1 ldRYl sc-bdVaJa hfjSkg')[0].text
        area_x = int(re.search(r'\d+', area_x).group())
        
        

cols = ['Rua', 'Bairro','Quartos','Area (m²)','Aluguel','Total']

dfApto = pd.DataFrame({
    'Rua' : rua,
    'Bairro' : bairro,
    'Quartos' : quartos,
    'Area' : area,
    'Aluguel' : aluguel,
    'Total' : total
})[cols]

In [3]:
#Cria Headers e envia HTTP Request para o site - conexão com o site 

browser = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}) # Mimic a browser

site =  'https://www.quintoandar.com.br/alugar/imovel/belo-horizonte-mg-brasil/apartamento/3-quartos/de-500-a-2500-aluguel/1-vagas'

response = requests.get(site, headers = browser)

response.status_code

200